In [1]:
from wetterdienst import Wetterdienst
from wetterdienst.provider.dwd.observation import DwdObservationRequest
import geopandas
import numpy as np
import pandas as pd

In [2]:
DWD = Wetterdienst(provider="dwd", network="observation")
request = DwdObservationRequest(
    parameter=["climate_summary"],
    resolution="daily",
    period="recent"
)

In [3]:
stations = request.all().df
data = request.all().values.all().df

In [4]:
stations = geopandas.GeoDataFrame(
    stations, geometry=geopandas.points_from_xy(stations.longitude, stations.latitude)
).drop(columns=["longitude", "latitude"]).rename(
    columns={"station_id": "id"}
).set_index(
    "id"
)
stations


,from_date,to_date,height,name,state,geometry
id,,,,,,
00011,1980-09-01 00:00:00+00:00,2020-09-30 00:00:00+00:00,680.0,Donaueschingen (Landeplatz),Baden-Württemberg,POINT (8.52050 47.97370)
00044,1969-01-01 00:00:00+00:00,2022-07-05 00:00:00+00:00,44.0,Großenkneten,Niedersachsen,POINT (8.23700 52.93360)
00071,1986-11-01 00:00:00+00:00,2019-12-31 00:00:00+00:00,759.0,Albstadt-Badkap,Baden-Württemberg,POINT (8.97840 48.21560)
00073,1959-03-01 00:00:00+00:00,2022-07-05 00:00:00+00:00,340.0,Aldersbach-Kriestorf,Bayern,POINT (13.05060 48.61590)
00078,1961-01-01 00:00:00+00:00,2022-07-05 00:00:00+00:00,65.0,Alfhausen,Niedersachsen,POINT (7.91260 52.48530)
...,...,...,...,...,...,...
15911,2021-06-01 00:00:00+00:00,2022-07-05 00:00:00+00:00,278.0,Großpostwitz-Denkwitz,Sachsen,POINT (14.43430 51.13480)
15976,2020-04-02 00:00:00+00:00,2022-07-05 00:00:00+00:00,216.0,Ober-Olm/Bellem,Rheinland-Pfalz,POINT (8.21070 49.95290)
15978,2020-04-02 00:00:00+00:00,2022-07-05 00:00:00+00:00,316.0,Sembach,Rheinland-Pfalz,POINT (7.86180 49.50440)


In [5]:
stations.to_file("src/assets/stations.geo.json", driver="GeoJSON")


In [14]:
len(data)

4351228

In [16]:
data

,station_id,dataset,parameter,date,value,quality
0,00011,climate_summary,wind_gust_max,2020-02-13 00:00:00+00:00,18.00,10.0
1,00011,climate_summary,wind_gust_max,2020-02-14 00:00:00+00:00,10.70,10.0
2,00011,climate_summary,wind_gust_max,2020-02-15 00:00:00+00:00,9.60,10.0
3,00011,climate_summary,wind_gust_max,2020-02-16 00:00:00+00:00,14.60,10.0
4,00011,climate_summary,wind_gust_max,2020-02-17 00:00:00+00:00,16.50,10.0
...,...,...,...,...,...,...
4351223,19172,climate_summary,temperature_air_min_005,2022-07-01 00:00:00+00:00,284.85,1.0
4351224,19172,climate_summary,temperature_air_min_005,2022-07-02 00:00:00+00:00,283.85,1.0
4351225,19172,climate_summary,temperature_air_min_005,2022-07-03 00:00:00+00:00,283.95,1.0
4351226,19172,climate_summary,temperature_air_min_005,2022-07-04 00:00:00+00:00,282.65,1.0


In [20]:
precipitation_form = {
    0: "no precipitation",
    1: "rain",
    4: "unknown",
    6: "rain",
    7: "snow",
    8: "rain and snow",
    9: "error or missing"
}

In [21]:
df = data.pivot(index=["station_id", "date"], columns="parameter", values="value")

In [30]:
df.precipitation_form = (
    df.precipitation_form.apply(
        lambda x: precipitation_form[int(x)] if not np.isnan(x) else x
    )
).astype("category")


In [42]:
for col in [col for col in list(df.columns) if col.startswith("temperature")] :
    df[col] = df[col] - 273.15

In [43]:
df

parameter                             cloud_cover_total  humidity  \
station_id date                                                     
00011      2020-02-13 00:00:00+00:00                NaN       NaN   
           2020-02-14 00:00:00+00:00                NaN       NaN   
           2020-02-15 00:00:00+00:00                NaN       NaN   
           2020-02-16 00:00:00+00:00                NaN       NaN   
           2020-02-17 00:00:00+00:00                NaN       NaN   
...                                                 ...       ...   
19172      2022-07-01 00:00:00+00:00                NaN     86.71   
           2022-07-02 00:00:00+00:00                NaN     68.08   
           2022-07-03 00:00:00+00:00                NaN     73.88   
           2022-07-04 00:00:00+00:00                NaN     76.33   
           2022-07-05 00:00:00+00:00                NaN     75.71   

parameter                            precipitation_form  precipitation_height  \
station_id date                                                                 
00011      2020-02-13 00:00:00+00:00                NaN                   NaN   
           2020-02-14 00:00:00+00:00                NaN                   NaN   
           2020-02-15 00:00:00+00:00                NaN                   NaN   
           2020-02-16 00:00:00+00:00                NaN                   NaN   
           2020-02-17 00:00:00+00:00                NaN                   NaN   
...                                                 ...                   ...   
19172      2022-07-01 00:00:00+00:00            unknown                   1.5   
           2022-07-02 00:00:00+00:00   no precipitation                   0.0   
           2022-07-03 00:00:00+00:00            unknown                   5.5   
           2022-07-04 00:00:00+00:00   no precipitation                   0.0   
           2022-07-05 00:00:00+00:00            unknown                   0.5   

parameter                             pressure_air_site  pressure_vapor  \
station_id date                                                           
00011      2020-02-13 00:00:00+00:00                NaN             NaN   
           2020-02-14 00:00:00+00:00                NaN             NaN   
           2020-02-15 00:00:00+00:00                NaN             NaN   
           2020-02-16 00:00:00+00:00                NaN             NaN   
           2020-02-17 00:00:00+00:00                NaN             NaN   
...                                                 ...             ...   
19172      2022-07-01 00:00:00+00:00                NaN          1640.0   
           2022-07-02 00:00:00+00:00                NaN          1360.0   
           2022-07-03 00:00:00+00:00                NaN          1530.0   
           2022-07-04 00:00:00+00:00                NaN          1370.0   
           2022-07-05 00:00:00+00:00                NaN          1310.0   

parameter                             snow_depth  sunshine_duration  \
station_id date                                                       
00011      2020-02-13 00:00:00+00:00         NaN                NaN   
           2020-02-14 00:00:00+00:00         NaN                NaN   
           2020-02-15 00:00:00+00:00         NaN                NaN   
           2020-02-16 00:00:00+00:00         NaN                NaN   
           2020-02-17 00:00:00+00:00         NaN                NaN   
...                                          ...                ...   
19172      2022-07-01 00:00:00+00:00         NaN                NaN   
           2022-07-02 00:00:00+00:00         NaN                NaN   
           2022-07-03 00:00:00+00:00         NaN                NaN   
           2022-07-04 00:00:00+00:00         NaN                NaN   
           2022-07-05 00:00:00+00:00         NaN                NaN   

parameter                             temperature_air_max_200  \
station_id date                                                 
00011      2020-02

In [50]:
df.to_csv("src/assets/data.csv", float_format="%g", date_format="%Y-%m-%d")